In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import sklearn

from pathlib import Path
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

In [2]:
# Configuraciones
data_path = '../../data/'
file_preffix = 'EXA_2022-1_'

In [3]:
# Cargamos todas las bases por departamento
states = ['AMAZONAS', 'GUAINIA', 'GUAVIARE', 'VAUPES', 'VICHADA']

bases = {}

for state in states:
    file_path = f'{data_path}raw/{file_preffix}{state}.csv'
    bases[state.lower()] = pd.read_csv(file_path, sep='|')

base_amazonas = bases['amazonas']
base_guainia = bases['guainia']
base_guaviare = bases['guaviare']
base_vaupes = bases['vaupes']
base_vichada = bases['vichada']

In [4]:
# Unificamos las bases en un solo dataframe
base = pd.concat([base_amazonas, base_guainia, base_guaviare, base_vaupes, base_vichada], ignore_index=True)
base

,ESTU_CONSECUTIVO,ESTU_FECHAPRESENTACION,ESTU_GRADO,ESTU_GENERO,COLE_COD_DANE_ESTABLECIMIENTO,COLE_COD_DANE_SEDE,COLE_NOM_ESTABLECIMIENTO,COLE_NOM_SEDE,COLE_COD_ICFES,COLE_NATURALEZA,...,EXA_INSTRUMENTO,EXA_COMPONENTE,EXA_COMPETENCIA,EXA_N_PREGUNTAS,EXA_N_PREGUNTAS_OM,EXA_PRC_PREGUNTAS_OM,EXA_N_RTAS_CORR,EXA_N_RTAS_NOCORR,EXA_PRC_RTAS_CORR,EXA_PRC_RTAS_NOCORR
0,22886860,30/05/2022,6,F,191001000012,191001000012,I.E. ESCUELA NORMAL SUPERIOR MARCELIANO EDUARD...,I.E. ESCUELA NORMAL SUPERIOR MARCELIANO EDUARD...,19505,O,...,Competencias Ciudadanas: Pensamiento Ciudadano,No aplica componente,Argumentación en contextos ciudadanos,4,0.0,0.0,0.0,4.0,0.00,100.00
1,22884841,23/05/2022,6,M,191001000012,191001000012,I.E. ESCUELA NORMAL SUPERIOR MARCELIANO EDUARD...,I.E. ESCUELA NORMAL SUPERIOR MARCELIANO EDUARD...,19505,O,...,Competencias Comunicativas en Lenguaje:Lectura,No aplica componente,Comprensión lectora,20,0.0,0.0,17.0,3.0,85.00,15.00
2,22888380,11/06/2022,5,M,191001000012,191001000047,I.E. ESCUELA NORMAL SUPERIOR MARCELIANO EDUARD...,SEDE VICENTE DE PAUL,19507,O,...,Ciencias Naturales y Educación Ambiental,Entorno vivo,Indagación,2,0.0,0.0,2.0,0.0,100.00,0.00
3,22972309,09/06/2022,8,M,191540000051,191540000051,I.E. INTERNADO SAN FRANCISCO DE LORETOYACO,I.E. INTERNADO SAN FRANCISCO DE LORETOYACO - S...,13116,O,...,Matemáticas,Aleatorio,Resolución de problemas,3,0.0,0.0,2.0,1.0,66.67,33.33
4,23572970,24/05/2022,3,M,191001000489,191001000489,INSTITUCION EDUCATIVA SAGRADO CORAZON DE JESUS,I.E. SAGRADO CORAZON DE JESUS - SEDE PRINCIPAL,13110,O,...,Matemáticas,Numérico Variacional,Comunicación,2,0.0,0.0,0.0,2.0,0.00,100.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
372557,22000211,27/05/2022,11,F,499001001919,499001001919,I.E. INTERNADO SANTA TERESITA DEL TUPARRO,I.E. INTERNADO SANTA TERESITA DEL TUPARRO - SE...,51930,O,...,Inglés,Parte 5,Nivel A2,2,2.0,100.0,0.0,0.0,0.00,0.00
372558,21999980,04/05/2022,7,M,299496000132,299496000132,I.E. INTERNADO CAMILO TORRES,I.E. INTERNADO CAMILO TORRES - SEDE PRINCIPAL,13044,O,...,Matemáticas,Aleatorio,Comunicación,2,0.0,0.0,0.0,2.0,0.00,100.00
372559,1330356,30/04/2022,10,F,299496000132,299496000132,I.E. INTERNADO CAMILO TORRES,I.E. INTERNADO CAMILO TORRES - SEDE PRINCIPAL,13044,O,...,Inglés,Parte 1,Nivel A1,1,0.0,0.0,1.0,0.0,100.00,0.00
372560,1326811,23/05/2022,9,F,299001001626,299001001626,I.E. INTERNADO SAN BARTOLOME,I.E. INTERNADO SAN BARTOLOME - SEDE PRINCIPAL,38674,O,...,Inglés,Parte 4,Nivel A1,1,0.0,0.0,1.0,0.0,100.00,0.00


In [5]:
# Obtener solo los registros del instrumento comprensión lectora
base_lec = base[base['EXA_COMPETENCIA']=='Comprensión lectora']

# Se eliminan los registros duplicados
base_lec_unique = base_lec.drop_duplicates(['ESTU_CONSECUTIVO'], keep='first')
base_lec_unique

,ESTU_CONSECUTIVO,ESTU_FECHAPRESENTACION,ESTU_GRADO,ESTU_GENERO,COLE_COD_DANE_ESTABLECIMIENTO,COLE_COD_DANE_SEDE,COLE_NOM_ESTABLECIMIENTO,COLE_NOM_SEDE,COLE_COD_ICFES,COLE_NATURALEZA,...,EXA_INSTRUMENTO,EXA_COMPONENTE,EXA_COMPETENCIA,EXA_N_PREGUNTAS,EXA_N_PREGUNTAS_OM,EXA_PRC_PREGUNTAS_OM,EXA_N_RTAS_CORR,EXA_N_RTAS_NOCORR,EXA_PRC_RTAS_CORR,EXA_PRC_RTAS_NOCORR
1,22884841,23/05/2022,6,M,191001000012,191001000012,I.E. ESCUELA NORMAL SUPERIOR MARCELIANO EDUARD...,I.E. ESCUELA NORMAL SUPERIOR MARCELIANO EDUARD...,19505,O,...,Competencias Comunicativas en Lenguaje:Lectura,No aplica componente,Comprensión lectora,20,0.0,0.0,17.0,3.0,85.0,15.0
9,22888972,27/05/2022,9,F,191001000012,191001000012,I.E. ESCUELA NORMAL SUPERIOR MARCELIANO EDUARD...,I.E. ESCUELA NORMAL SUPERIOR MARCELIANO EDUARD...,19502,O,...,Competencias Comunicativas en Lenguaje:Lectura,No aplica componente,Comprensión lectora,20,0.0,0.0,8.0,12.0,40.0,60.0
13,22861481,22/05/2022,10,F,191001000012,191001000012,I.E. ESCUELA NORMAL SUPERIOR MARCELIANO EDUARD...,I.E. ESCUELA NORMAL SUPERIOR MARCELIANO EDUARD...,19502,O,...,Lectura Crítica,No aplica componente,Comprensión lectora,20,0.0,0.0,10.0,10.0,50.0,50.0
14,22970239,15/06/2022,5,F,191540000051,191540000051,I.E. INTERNADO SAN FRANCISCO DE LORETOYACO,I.E. INTERNADO SAN FRANCISCO DE LORETOYACO - S...,13116,O,...,Competencias Comunicativas en Lenguaje:Lectura,No aplica componente,Comprensión lectora,20,0.0,0.0,10.0,10.0,50.0,50.0
83,22891079,22/05/2022,10,F,191001000012,191001000012,I.E. ESCUELA NORMAL SUPERIOR MARCELIANO EDUARD...,I.E. ESCUELA NORMAL SUPERIOR MARCELIANO EDUARD...,19502,O,...,Lectura Crítica,No aplica componente,Comprensión lectora,20,0.0,0.0,8.0,12.0,40.0,60.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
372443,21990399,25/05/2022,5,F,299524000194,299524000194,C.E. EL TROMPILLO,C.E. EL TROMPILLO - SEDE PRINCPAL,6640,O,...,Competencias Comunicativas en Lenguaje:Lectura,No aplica componente,Comprensión lectora,20,0.0,0.0,6.0,14.0,30.0,70.0
372444,1293291,11/05/2022,6,F,199001001974,199001001974,I.E. SOLMERIDA BUILES,I.E. SOLMERIDA BUILES - SEDE PRINCIPAL,13029,O,...,Competencias Comunicativas en Lenguaje:Lectura,No aplica componente,Comprensión lectora,20,0.0,0.0,16.0,4.0,80.0,20.0
372454,21994388,01/06/2022,4,F,299572000082,299001002126,I.E. INTERNADO CADANAPAY,TARRAPAY,13069,O,...,Competencias Comunicativas en Lenguaje:Lectura,No aplica componente,Comprensión lectora,20,0.0,0.0,7.0,13.0,35.0,65.0
372456,21990395,27/05/2022,7,M,299524000194,299524000194,C.E. EL TROMPILLO,C.E. EL TROMPILLO - SEDE PRINCPAL,6640,O,...,Competencias Comunicativas en Lenguaje:Lectura,No aplica componente,Comprensión lectora,20,0.0,0.0,18.0,2.0,90.0,10.0


In [6]:
# Se verifica que no exista más de 1 registro por estudiante
base_lec_unique.groupby(['ESTU_CONSECUTIVO'])['ESTU_CONSECUTIVO'].filter(lambda x: len(x) > 1)

Series([], Name: ESTU_CONSECUTIVO, dtype: int64)

In [7]:
# Sumar numero de respuestas por instrumento y fecha
base_grouped = base.groupby(['ESTU_CONSECUTIVO', 'ESTU_FECHAPRESENTACION', 'EXA_INSTRUMENTO'])['EXA_N_RTAS_CORR'].sum()
base_grouped = pd.DataFrame(base_grouped).reset_index().sort_values(by=['ESTU_CONSECUTIVO', 'ESTU_FECHAPRESENTACION', 'EXA_INSTRUMENTO'])
base_grouped

,ESTU_CONSECUTIVO,ESTU_FECHAPRESENTACION,EXA_INSTRUMENTO,EXA_N_RTAS_CORR
0,25622,27/05/2022,Ciencias Naturales y Educación Ambiental,8.0
1,25622,27/05/2022,Competencias Comunicativas en Lenguaje:Lectura,16.0
2,118033,18/05/2022,Matemáticas,14.0
3,118033,25/05/2022,Ciencias Naturales,12.0
4,118033,25/05/2022,Lectura Crítica,6.0
...,...,...,...,...
66644,23931572,19/06/2022,Matemáticas,7.0
66645,23932246,08/05/2022,Competencias Comunicativas en Lenguaje:Lectura,4.0
66646,23932246,09/05/2022,Matemáticas,6.0
66647,23932418,04/05/2022,Competencias Comunicativas en Lenguaje:Lectura,28.0


In [8]:
# Sumar la cantidad de respuestas correctas por instrumento por estudiante
base_columns = ['ESTU_CONSECUTIVO', 'ESTU_FECHAPRESENTACION']

base_pivot = pd.pivot_table(base_grouped,
                             index=['ESTU_CONSECUTIVO'],
                             columns=["EXA_INSTRUMENTO"],
                             values=["EXA_N_RTAS_CORR"],
                             aggfunc="sum",
                             fill_value=0)

base_pivot.columns = base_pivot.columns.droplevel()
pivoted = base_pivot.groupby(['ESTU_CONSECUTIVO']).max().sort_values(by=['ESTU_CONSECUTIVO']).reset_index()
pivoted

EXA_INSTRUMENTO,ESTU_CONSECUTIVO,Ciencias Naturales,Ciencias Naturales y Educación Ambiental,Competencias Ciudadanas: Pensamiento Ciudadano,Competencias Comunicativas en Lenguaje:Lectura,Inglés,Lectura Crítica,Matemáticas,Sociales y Ciudadanas
0,25622,0.0,8.0,0.0,16.0,0.0,0.0,0.0,0.0
1,118033,12.0,0.0,0.0,0.0,12.0,6.0,14.0,8.0
2,196762,0.0,2.0,2.0,3.0,0.0,0.0,3.0,0.0
3,196957,0.0,6.0,12.0,17.0,0.0,0.0,12.0,0.0
4,396680,15.0,0.0,0.0,0.0,15.0,9.0,7.0,12.0
...,...,...,...,...,...,...,...,...,...
19648,23930490,0.0,0.0,0.0,9.0,0.0,0.0,5.0,0.0
19649,23931074,0.0,0.0,0.0,12.0,0.0,0.0,8.0,0.0
19650,23931572,0.0,0.0,0.0,11.0,0.0,0.0,7.0,0.0
19651,23932246,0.0,0.0,0.0,4.0,0.0,0.0,6.0,0.0


In [9]:
# Hacemos merge de la base inicial de instrumento de lectura por estudiante con la sumatoria de respuestas correctas por otro instrumentos.
summarized = pd.merge(base_lec_unique, pivoted, on='ESTU_CONSECUTIVO', how='left')
summarized = summarized.sort_values(by=['COLE_NOM_DPTO', 'COLE_COD_ICFES', 'ESTU_CONSECUTIVO', 'ESTU_FECHAPRESENTACION', 'EXA_INSTRUMENTO'])
summarized.to_csv(f'{data_path}processed/{file_preffix}Todos.csv')
summarized

,ESTU_CONSECUTIVO,ESTU_FECHAPRESENTACION,ESTU_GRADO,ESTU_GENERO,COLE_COD_DANE_ESTABLECIMIENTO,COLE_COD_DANE_SEDE,COLE_NOM_ESTABLECIMIENTO,COLE_NOM_SEDE,COLE_COD_ICFES,COLE_NATURALEZA,...,EXA_PRC_RTAS_CORR,EXA_PRC_RTAS_NOCORR,Ciencias Naturales,Ciencias Naturales y Educación Ambiental,Competencias Ciudadanas: Pensamiento Ciudadano,Competencias Comunicativas en Lenguaje:Lectura,Inglés,Lectura Crítica,Matemáticas,Sociales y Ciudadanas
332,22857330,26/05/2022,3,M,191001000489,191001000489,INSTITUCION EDUCATIVA SAGRADO CORAZON DE JESUS,I.E. SAGRADO CORAZON DE JESUS - SEDE PRINCIPAL,13110,O,...,40.0,60.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0
1758,22857841,24/05/2022,3,F,191001000489,191001000489,INSTITUCION EDUCATIVA SAGRADO CORAZON DE JESUS,I.E. SAGRADO CORAZON DE JESUS - SEDE PRINCIPAL,13110,O,...,30.0,70.0,0.0,0.0,0.0,6.0,0.0,0.0,10.0,0.0
526,22861931,26/05/2022,3,F,191001000489,191001000489,INSTITUCION EDUCATIVA SAGRADO CORAZON DE JESUS,I.E. SAGRADO CORAZON DE JESUS - SEDE PRINCIPAL,13110,O,...,40.0,60.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0
1902,22884935,24/05/2022,3,F,191001000489,191001000489,INSTITUCION EDUCATIVA SAGRADO CORAZON DE JESUS,I.E. SAGRADO CORAZON DE JESUS - SEDE PRINCIPAL,13110,O,...,35.0,65.0,0.0,0.0,0.0,7.0,0.0,0.0,10.0,0.0
2505,22884936,26/05/2022,3,M,191001000489,191001000489,INSTITUCION EDUCATIVA SAGRADO CORAZON DE JESUS,I.E. SAGRADO CORAZON DE JESUS - SEDE PRINCIPAL,13110,O,...,70.0,30.0,0.0,0.0,0.0,14.0,0.0,0.0,12.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14013,22000750,31/05/2022,4,F,299773002598,299773000498,C.E. INTERNADO LA LIBERTAD,SAN ERNESTO DEL TUPARRO,67047,O,...,25.0,75.0,0.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0
16635,22000751,31/05/2022,5,F,299773002598,299773000498,C.E. INTERNADO LA LIBERTAD,SAN ERNESTO DEL TUPARRO,67047,O,...,20.0,80.0,0.0,4.0,5.0,4.0,0.0,0.0,11.0,0.0
16478,22000752,31/05/2022,4,M,299773002598,299773000498,C.E. INTERNADO LA LIBERTAD,SAN ERNESTO DEL TUPARRO,67047,O,...,20.0,77.5,0.0,0.0,0.0,8.0,0.0,0.0,5.0,0.0
14591,23347452,31/05/2022,3,F,299773002598,299773000498,C.E. INTERNADO LA LIBERTAD,SAN ERNESTO DEL TUPARRO,67047,O,...,30.0,70.0,0.0,0.0,0.0,6.0,0.0,0.0,7.0,0.0
